### Quick start

In [ ]:
%cd /content
!git clone https://github.com/NTT123/vietTTS.git
%cd /content/vietTTS
!pip3 install -e .
!bash ./scripts/quick_start.sh
from google.colab import output
output.clear()

In [ ]:
text = """
Đoạn trường tân thanh, thường được biết đến với cái tên đơn giản là Truyện Kiều, là một truyện thơ của đại thi hào Nguyễn Du. 
Đây được xem là truyện thơ nổi tiếng nhất và xét vào hàng kinh điển trong văn học Việt Nam.
Câu chuyện dựa theo tiểu thuyết Kim Vân Kiều truyện của Thanh Tâm Tài Nhân, một thi sĩ thời nhà Minh, Trung Quốc.
Tác phẩm kể lại cuộc đời, những thử thách và đau khổ của Thúy Kiều, một phụ nữ trẻ xinh đẹp và tài năng, phải hy sinh thân mình để cứu gia đình. 
Để cứu cha và em trai khỏi tù, cô bán mình kết hôn với một người đàn ông trung niên, không biết rằng anh ta là một kẻ buôn người, và bị ép làm kĩ nữ trong lầu xanh.
"""
!python3 -m vietTTS.synthesizer --lexicon-file assets/infore/lexicon.txt --text="{text}" --output=clip.wav --silence-duration 0.2
import IPython.display as ipd
ipd.Audio('clip.wav')

Normalized text input: đoạn trường tân thanh sil thường được biết đến với cái tên đơn giản là truyện kiều sil là một truyện thơ của đại thi hào nguyễn du sil đây được xem là truyện thơ nổi tiếng nhất và xét vào hàng kinh điển trong văn học việt nam sil câu chuyện dựa theo tiểu thuyết kim vân kiều truyện của thanh tâm tài nhân sil một thi sĩ thời nhà minh sil trung quốc sil tác phẩm kể lại cuộc đời sil những thử thách và đau khổ của thúy kiều sil một phụ nữ trẻ xinh đẹp và tài năng sil phải hy sinh thân mình để cứu gia đình sil để cứu cha và em trai khỏi tù sil cô bán mình kết hôn với một người đàn ông trung niên sil không biết rằng anh ta là một kẻ buôn người sil và bị ép làm kĩ nữ trong lầu xanh sil
/usr/local/lib/python3.7/dist-packages/jax/_src/tree_util.py:189: FutureWarning: jax.tree_util.tree_multimap() is deprecated. Please use jax.tree_util.tree_map() instead as a drop-in replacement.
  'instead as a drop-in replacement.', FutureWarning)
writing output to file clip.wav


# Train duration model and acoustic model

In [ ]:
!bash ./scripts/download_aligned_infore_dataset.sh

/content/vietTTS /content/vietTTS
Downloading...
From: https://drive.google.com/uc?id=1Pe-5lKT_lZsliv2WxQDai2mjhI9ZMFlj
To: /content/vietTTS/train_data/infore.zip
100% 2.44G/2.44G [00:12<00:00, 189MB/s]
/content/vietTTS


In [ ]:
!python3 -m vietTTS.nat.duration_trainer

In [ ]:
!python3 -m vietTTS.nat.acoustic_trainer

---------------------------  -------------------------
duration_lstm_dim            256
vocab_size                   256
duration_embed_dropout_rate  0.5
num_training_steps           200000
postnet_dim                  512
acoustic_decoder_dim         512
acoustic_encoder_dim         256
max_phoneme_seq_len          384
max_wave_len                 196608
special_phonemes             ['sil', 'sp', 'spn', ' ']
sil_index                    0
sp_index                     1
word_end_index               3
mel_dim                      80
n_fft                        1024
sample_rate                  16000
fmin                         0.0
fmax                         8000
batch_size                   64
learning_rate                0.0001
duration_learning_rate       0.0001
max_grad_norm                1.0
weight_decay                 0.0001
ckpt_dir                     assets/infore/nat
data_dir                     train_data
---------------------------  -------------------------
Resuming fr

# Train HiFiGAN vocoder

In [ ]:
%cd /content/vietTTS
!bash ./scripts/download_aligned_infore_dataset.sh


/content/vietTTS
/content/vietTTS /content/vietTTS
Downloading...
From: https://drive.google.com/uc?id=1Pe-5lKT_lZsliv2WxQDai2mjhI9ZMFlj
To: /content/vietTTS/train_data/infore.zip
2.44GB [00:21, 115MB/s]
replace 00945.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!git clone https://github.com/jik876/hifi-gan.git
%cd hifi-gan
!ln -sf /content/vietTTS/train_data /content/vietTTS/hifi-gan/data
!cd data;  ls -1 *.TextGrid| sed -e 's/\.TextGrid$//' > /content/vietTTS/hifi-gan/files.txt
!head -n 100 ./files.txt > val_files.txt
!tail -n +101 files.txt > train_files.txt
!cp /content/vietTTS/assets/hifigan/config.json ./config.json

In [ ]:
!python3 train.py --config config.json --input_wavs_dir=data  --input_training_file=train_files.txt  --input_validation_file=val_files.txt